In [1]:
from shapely.geometry import Point
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster
import xlrd
import time
from IPython.display import Audio

In [2]:
#full runtime indicator start
full_time=time.time()

In [3]:
#finish indicator
wow = '../../Data/Wow.mp3'
finished=Audio(url = wow, autoplay = True) 

In [4]:
#notebook complete indicator
notebook = '../../Data/Hugo Good Job.mp3'
complete=Audio(url = notebook, autoplay = True)

In [5]:
map = gpd.read_file('../../Data/Taxi Zones Shape(original, do not use)/taxi_zones.shp').to_crs('epsg:4326')
map = map.drop(['Shape_Area', 'Shape_Leng', 'OBJECTID'], axis=1)


complete

In [6]:
# Double check before running agin


In [7]:
df=pd.read_csv('../../Data/Taxi Data/yellow_15/yellow_tripdata_2015-06.csv')

In [8]:
df.columns

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'pickup_longitude',
       'pickup_latitude', 'RateCodeID', 'store_and_fwd_flag',
       'dropoff_longitude', 'dropoff_latitude', 'payment_type', 'fare_amount',
       'extra', 'mta_tax', 'tip_amount', 'tolls_amount',
       'improvement_surcharge', 'total_amount'],
      dtype='object')

df.info()

df2=df.head(100)

df2.columns

In [9]:
start_time = time.time()

df['dropoff_geometry'] = df.apply(lambda x: Point((float(x['dropoff_longitude']), 
                                             float(x['dropoff_latitude']))), 
                                             axis=1)

print("--- %s seconds ---" % (time.time() - start_time))
finished

--- 750.1369128227234 seconds ---


In [10]:
start_time = time.time()

df['pickup_geometry'] = df.apply(lambda x: Point((float(x['pickup_longitude']), 
                                             float(x['pickup_latitude']))), 
                                             axis=1)

print("--- %s seconds ---" % (time.time() - start_time))
finished

--- 754.5659999847412 seconds ---


In [11]:
df.head(3)

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,store_and_fwd_flag,dropoff_longitude,...,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,dropoff_geometry,pickup_geometry
0,2,2015-06-02 11:19:29,2015-06-02 11:47:52,1,1.63,-73.954430,40.764141,1,N,-73.974754,...,2,17.0,0.0,0.5,0.0,0.0,0.3,17.8,POINT (-73.97475433349609 40.75409317016602),POINT (-73.95442962646483 40.76414108276367)
1,2,2015-06-02 11:19:30,2015-06-02 11:27:56,1,0.46,-73.971443,40.758942,1,N,-73.978539,...,1,6.5,0.0,0.5,1.0,0.0,0.3,8.3,POINT (-73.97853851318358 40.76190948486328),POINT (-73.97144317626953 40.75894165039063)
2,2,2015-06-02 11:19:31,2015-06-02 11:30:30,1,0.87,-73.978111,40.738434,1,N,-73.990273,...,1,8.0,0.0,0.5,2.2,0.0,0.3,11.0,POINT (-73.99027252197266 40.74543762207031),POINT (-73.97811126708984 40.73843383789063)


In [12]:
df2_start=df[['passenger_count','trip_distance','fare_amount','mta_tax','tip_amount','tolls_amount','pickup_geometry']]

In [13]:
df2_end=df[['dropoff_geometry']]

In [14]:
start_time = time.time()
df2_start=gpd.GeoDataFrame(df2_start,
                     crs=map.crs,
                     geometry=df2_start['pickup_geometry'])
print("--- %s seconds ---" % (time.time() - start_time))
finished

--- 102.79178428649902 seconds ---


In [15]:
start_time = time.time()
df2_end=gpd.GeoDataFrame(df2_end,
                     crs=map.crs,
                     geometry=df2_end['dropoff_geometry'])
print("--- %s seconds ---" % (time.time() - start_time))
finished

--- 103.82260251045227 seconds ---


In [16]:
start_time = time.time()

geo_processing_start = gpd.sjoin(df2_start, map, op = 'within')

print("--- %s seconds ---" % (time.time() - start_time))
finished

--- 1014.8253664970398 seconds ---


In [17]:
start_time = time.time()

geo_processing_end = gpd.sjoin(df2_end, map, op = 'within')

print("--- %s seconds ---" % (time.time() - start_time))
finished

--- 1002.3548064231873 seconds ---


In [18]:
geo_processing_start

,passenger_count,trip_distance,fare_amount,mta_tax,tip_amount,tolls_amount,pickup_geometry,geometry,index_right,zone,LocationID,borough
0,1,1.63,17.0,0.5,0.00,0.00,POINT (-73.95442962646483 40.76414108276367),POINT (-73.95443 40.76414),139,Lenox Hill East,140,Manhattan
13,2,0.50,4.5,0.5,1.30,0.00,POINT (-73.95480346679686 40.76566696166992),POINT (-73.95480 40.76567),139,Lenox Hill East,140,Manhattan
51,1,1.70,11.5,0.5,0.00,0.00,POINT (-73.95548248291014 40.76830291748047),POINT (-73.95548 40.76830),139,Lenox Hill East,140,Manhattan
93,5,1.90,14.5,0.5,3.06,0.00,POINT (-73.95390319824217 40.76681900024414),POINT (-73.95390 40.76682),139,Lenox Hill East,140,Manhattan
112,2,3.85,15.0,0.5,0.00,0.00,POINT (-73.9510040283203 40.77059555053711),POINT (-73.95100 40.77060),139,Lenox Hill East,140,Manhattan
...,...,...,...,...,...,...,...,...,...,...,...,...
7596886,1,4.00,13.0,0.5,0.00,0.00,POINT (-73.91856384277344 40.56090545654297),POINT (-73.91856 40.56091),26,Breezy Point/Fort Tilden/Riis Beach,27,Queens
10756897,1,11.89,34.5,0.5,0.00,2.08,POINT (-73.9246368408203 40.55968856811523),POINT (-73.92464 40.55969),26,Breezy Point/Fort Tilden/Riis Beach,27,Queens
10891228,6,1.43,15.0,0.5,0.00,0.00,POINT (-73.91120910644531 40.56090927124024),POINT (-73.91121 40.56091),26,Breezy Point/Fort Tilden/Riis Beach,27,Queens
10898212,6,0.43,7.0,0.5,0.00,0.00,POINT (-73.91120910644531 40.56090927124024),POINT (-73.91121 40.56091),26,Breezy Point/Fort Tilden/Riis Beach,27,Queens


In [19]:
group_start = geo_processing_start.groupby(['LocationID']).describe()

In [20]:
group_start.columns = ['_'.join(col) for col in group_start.columns.values]

group_start.to_csv('../../Data/yellow_tripdata_2014-2-start-processed.csv')

In [21]:
group_end = geo_processing_end.value_counts(['LocationID'])

In [22]:
group_end = pd.DataFrame(group_end,columns=['dropoff_count'])

group_end.to_csv('../../Data/yellow_tripdata_2014-2-end-processed.csv')

In [23]:
merge=group_start.merge(group_end, how='left', on='LocationID')

In [24]:
merge

,passenger_count_count,passenger_count_mean,passenger_count_std,passenger_count_min,passenger_count_25%,passenger_count_50%,passenger_count_75%,passenger_count_max,trip_distance_count,trip_distance_mean,...,tolls_amount_max,index_right_count,index_right_mean,index_right_std,index_right_min,index_right_25%,index_right_50%,index_right_75%,index_right_max,dropoff_count
LocationID,,,,,,,,,,,,,,,,,,,,,
1,706.0,1.422096,0.885033,0.0,1.0,1.0,2.0,9.0,706.0,2.350567,...,145.00,706.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19189
2,16.0,1.437500,1.030776,1.0,1.0,1.0,1.0,4.0,16.0,7.630000,...,5.54,16.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,10
3,44.0,1.431818,1.189053,1.0,1.0,1.0,1.0,6.0,44.0,5.553636,...,11.75,44.0,2.0,0.0,2.0,2.0,2.0,2.0,2.0,744
4,38551.0,1.672745,1.324628,0.0,1.0,1.0,2.0,6.0,38551.0,2.816257,...,22.83,38551.0,3.0,0.0,3.0,3.0,3.0,3.0,3.0,62073
5,2.0,2.000000,0.000000,2.0,2.0,2.0,2.0,2.0,2.0,0.000000,...,0.00,2.0,4.0,0.0,4.0,4.0,4.0,4.0,4.0,49
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259,75.0,1.600000,1.252025,1.0,1.0,1.0,2.0,6.0,75.0,3.736533,...,5.54,75.0,258.0,0.0,258.0,258.0,258.0,258.0,258.0,1528
260,7630.0,1.735518,1.437302,1.0,1.0,1.0,2.0,6.0,7630.0,3.280177,...,40.00,7630.0,259.0,0.0,259.0,259.0,259.0,259.0,259.0,16046
261,65349.0,1.728932,1.332438,0.0,1.0,1.0,2.0,7.0,65349.0,4.089857,...,29.70,65349.0,260.0,0.0,260.0,260.0,260.0,260.0,260.0,63315


In [25]:
#REMEMBER TO RENAME CSV BEFORE START
merge.to_csv('../../Data/yellow_tripdata_2015-06-processed.csv')
complete

In [26]:
print("--- %s seconds ---" % (time.time() - full_time))

--- 3771.038571834564 seconds ---
